In [ ]:
# Создание страниц

In [ ]:
# Создадим еще одну модель - класс Жанры

class Genre(models.Model):
    '''жанры фильмов'''
    name = models.CharField('Название жанра', max_length=50)
    description = models.TextField('Описание', blank=True)
    url = models.SlugField(max_length=50, unique=True) # SlugField отвечает за адрес страницы, unique - уникальная
    
    def __str__(self): # Отображает в админ панели название, а не Object1, Object1 и т.д
        return self.name
    
     class Meta:  # Отображает название на русском языке
        verbose_name = 'Жанр'
        verbose_name_plural = 'Жанры'
        
# Точно также создаем очередную модель - класс Режиссеры

class Direction(models.Model):
    '''Режиссеры'''
    name = models.CharField('Фамилия и Имя', max_length=100)
    description = models.TextField('Биография', blank=True)
    image = models.ImageField('Фотография', upload_to='image/directions/%Y') # создаем папку directions и сортируем по дате %Y

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = 'Режиссер'
        verbose_name_plural = 'Режиссеры'
        
# В модель Film добавим: 
directions = models.ManyToManyField(Direction, verbose_name='Режиссеры')
genre = models.ManyToManyField(Genre, verbose_name='Жанры')

# ManyToManyField(многие ко многим) связывает модели Фильмы и Режиссеры. В данном случае по таблице Direction и Genre

class Film(models.Model): # Создаем класс Film
    '''Информация о фильме'''
    title = models.CharField(max_length = 100) # Поле для названия
    imag = models.ImageField(upload_to='image/&Y') # Поле для картинки(&Y - авто-сортировка в папке по годам)
    description = models.TextField(blank=True) # Поле для краткого описания(blanc=True - поле не обязательно для заполнения)
    date_publ = models.DateField('Дата выхода')
    directions = models.ManyToManyField(Direction, verbose_name='Режиссеры')
    genre = models.ManyToManyField(Genre, verbose_name='Жанры')
    url = models.SlugField(max_length='100', unique=True) # У каждого фильма должна быть своя уникальная страница

In [ ]:
# Создаем миграции: python manage.py makemigrations
# Если возникает ошибка: 
    Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows with a null value for this column)
 2) Quit and manually define a default value in models.py.

Select an option: 1
Please enter the default value as valid Python.
    
# Выбираем 1, нажимаем Enter.

The datetime and django.utils.timezone modules are available, so it is possible to provide e.g. timezone.now as a value.
Type 'exit' to exit this prompt

# Нужно прописать произвольный адрес URL в типе STR, например 'comedy'

# Когда будем создавать новый фильм, то пропишем ему свой адрес

# Применяем миграции: python manage.py migration

# Импортируем эти модели в файле admin.py: from .models import Film, (дописываем их в строку)Genre, Direction

# Регистрируем созданные модели в файле admin.py:

@admin.register(Genre)
class GenreAdmin(admin.ModelAdmin):
    list_display = ('name', )

@admin.register(Direction)
class DirectionAdmin(admin.ModelAdmin):
    list_display = ('name', )

In [ ]:
# Запускаем сайт: python manage.py runserver
# В админ-панели добавляем жанры, фильмы и их режиссеров
# В папке Films создаем два файла: urls.py и form.py - они будут относится только к приложению Films
# В основной категории Mysite сосдаем папку шаблоны - templates

In [ ]:
# Схема написания представления(MVT): написать представление - подключить HTML шаблон - передать в него данные из базы данных
# Представления пишем в файле views.py
# Импортируем родительский класс View: from django.views.generic.base import View
# Импортируем из нашего файла models.py класс, который описывает поведение данной модели: from .models import Film
# Создаем класс - представление фильмов:

class FilmsView(View):
    '''Список фильмов'''
    def get(self, request): # внутри класса определяем метод с обязательным именем get
        films = Film.objects.all() # в переменную films помещается ссылка на все(all) записи о модели Film
        return render(request, 'films/film.html', {'film_list': films}) # возвращает запрос пользователя, ссылку на шаблон и контекст в виде словаря
        # render - функция, которая объединяет шаблон со словарем и возвращает объект с отображаемым текстом
        

In [ ]:
# В папке templates создаем папку films, а в ней html файл film.html
# В HTML шаблон код Python встраивается через {%  %}

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Title</title>
</head>
<body>

{% for film in film_list %} # запускаем цикл в списке всех фильмов
<H2>Названия фильмов: {{ film.title }}</H2> # выводим названия всех фильмов из списка
<p>Описание фильма: {{ film.description }}</p> # выводим названия описание фильмов

<p>
    {% for direction in film.directions.all %}  # Добавляем режиссеров
       {{ direction.name }}
    {% endfor %}
</p>

<p>
    {% for direction in film.directions.all %} # Добавляем жанр
        {{ direction.name }}
    {% endfor %}
</p>

<p>Постер: </p>
<img src="{{ film.imag.url }}" width="150px" height="150px"> # Вставляем картинку-постер
{% endfor %} # обязательно закрываем цикл

</body>
</html>

In [ ]:
# Теперь подключаем папку templates в файле settings.py в разделе 'TEMPLATES'
# в строке 'DIRS': [] прописываем путь к папке templates - 'DIRS': [os.path.join(BASE_DIR, 'templates')]

In [ ]:
# В файле urls.py из папки Films нужно прописать адрес главной страницы

from django.urls import path # импортируем модуль path(путь)
from . import views # импортируем наши представления из файла views

# При заходе на главную страницу('') пользователю будут генерироваться представления FilmsView(файл views.py):

urlpatterns = [
    path('', views.FilmsView.as_view())
]

# Теперь нужно в главном файле urls.py(Mysite) подключить файл urls.py из папки Films

# Импортируем функцию include (from django.urls import path, include)

# Строку path('', admin.site.urls) меняем на path('', include('films.urls'))

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('films.urls'))
]

# Запускаем сервер для теста: python manage.py runserver


In [ ]:
# Добавляем ссылку на фильм на странице

# В папке films создаем HTML файл film_detail.html

# Копируем весь код между тегами <body> из файла film.html в файл film_detail.html

# Удаляем цикла {% for film in film_list %} {% endfor %}, т.к страница film_detail.html для одного фильма, выводить все не нужно

# Теперь нужно написать в файле views.py представление для этой страницы одного фильма - что будет на ней выводится:

class FilmDetail(View):
    '''Представление одного фильма'''
    def get(self, request, slug): # Помимо запроса пользователя(request) функция получает конкретный(slug) url-адрес фильма
        film = Film.objects.get(url=slug)
        return render(request, 'films/film_detail.html', {'film': film}) # передаем функции запрос и путь к странице

# В файле urls.py(папка Films) в urlpatterns добавить представление, которое генерируется при заходе на конкретную страницу

# path('<slug:slug>/', views.FilmDetail.as_view())

urlpatterns = [
    path('', views.FilmsView.as_view()),
    path('<slug:slug>/', views.FilmDetail.as_view())
]

In [ ]:
# Размещаем ссылки на страницы с фильмами на главной странице

# На странице film.html в строку <H2>Названия фильмов: {{ film.title }}</H2> добавим <a href="/{{ film.url }}/"> </a>

<h2><a href="/{{ film.url }}/"> {{ film.title }}</a> </h2>